In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
#pip install smogn
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use('ggplot')


from pandas.plotting import scatter_matrix
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_regression, RFECV
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.model_selection import train_test_split,StratifiedShuffleSplit, KFold

from sklearn.linear_model import LinearRegression, RidgeCV,LassoCV,ElasticNetCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

import statsmodels.api as sm
from statsmodels.compat import lzip
import statsmodels.stats.api as sms
from statsmodels.formula.api import ols
from scipy.stats import zscore
from statsmodels.stats.stattools import durbin_watson
from statsmodels.stats.outliers_influence import variance_inflation_factor
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

df =  pd.read_csv(os.path.join(dirname, filenames[0]))
df.columns
# data = np.loadtxt("/kaggle/input/forest-fires-data-set/forestfires.csv", delimiter=',')
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df.info() 


In [ ]:
 # Generate descriptive statistics that summarize the central tendency,dispersion and shape of a dataset's distribution
df.describe()

# Handling Missing Values

Now we randomly simulate the presence of missing values by making a good chunk of our data as missing

In [ ]:
from numpy.random import default_rng
rng = default_rng()
print(f"Before missing values injection {df.isnull().sum()}")
cols_with_missing_data = ['wind', 'RH', 'DMC', 'ISI', 'rain']
missing_indices_rain = rng.choice(len(df['rain'])-1, size=int(0.6 * len(df['rain'])), replace=False)
missing_indices = list(map(lambda col: rng.choice(len(df[col])-1, size=int(0.15 * len(df[col])), replace=False), cols_with_missing_data[0:-1]))
missing_indices.append(missing_indices_rain)

for col in range(len(cols_with_missing_data)):
    for missingIndex in missing_indices[col]:
        df[cols_with_missing_data[col]][missingIndex] = np.nan

print(f"After missing values injection {df.isnull().sum()}")

df[1:20]

# Imputing missing values using the most performant way

Using SimpleImputer with all strategies

In [ ]:

def SimpleImputerImpute(X, y):
    results = []
    strategies = ['mean','median','most_frequent','constant']
    for s in strategies:

        split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

        for train_index, test_index in split.split(df.values, df.temp_bins.values):
            st_train_set = df.iloc[train_index]
            st_test_set = df.iloc[test_index]

        imputer = SimpleImputer(strategy=s)
        X_train = st_train_set.drop(columns=['area', 'temp_bins', 'damage_category'])
        imputer.fit(X_train)
        X_train = imputer.transform(X_train)
        X_train = pd.get_dummies(X_train)
        y_train = st_train_set[['damage_category']]

        X_test = st_test_set.drop(columns=['area', 'temp_bins', 'damage_category'])
        imputer.fit(X_test)
        X_test = imputer.transform(X_test)
        X_test = pd.get_dummies(X_test)
        y_test = st_test_set[['damage_category']]

        model = RandomForestRegressor(max_depth=2, min_samples_split=2, min_samples_leaf=1, random_state=0, n_estimators=100)


        model.fit(X_train, y_train.values.ravel())

        y_pred = model.predict(X_test)
        # print out the prediction scores
        print("------------------------------------------------------------")
        print('RMSE: {}'.format(np.sqrt(mean_squared_error(y_test, y_pred))))
        print('MAE: {}'.format(mean_absolute_error(y_test, y_pred)))
        print('R-squared: {}'.format(r2_score(y_test, y_pred)))
        print("------------------------------------------------------------")

dfa = pd.get_dummies(df)
data = dfa.values
cols = dfa.columns
ix = [i for i in range(data.shape[1]) if  i != 10]

X_train = data[:, ix] 
y_train = data[:, 10]


from sklearn.utils.multiclass import type_of_target
type_of_target(df['area'].values)

list(SimpleImputerImpute(X_train, y_train))

SimpleImputerImpute(X_train, )

Using KNN imputer with and choosing the best features numbers/combinations

In [ ]:

def KNNImpute(X, y):
    results = []
    strategies = [1,3,5,7,8,9,15,18,21,25]
    for s in strategies:
        pipeline = Pipeline(steps=[('i', KNNImputer(n_neighbors=int(s))), ('m', RandomForestClassifier())] )


        #evaluate model 
        cv = RepeatedStratifiedKFold(n_splits = 10,  n_repeats=3, random_state= 1)
        scores = cross_val_score(pipeline, X , y ,scoring='accuracy', cv=cv, n_jobs=-1)

        #store results 
        results.append(np.mean(scores))
        
        print('>%s %.3f (%.3f)'% (s, np.mean(scores), np.std(scores)))
        if(s == 25):
            strat = strategies[index(max(results))]
            imputer = KNNImputer(n_neighbors=int(strat))
            X = imputer.fit_transform(X)
            df = pd.DataFrame(data= X)
            
        
KNNImpute(X_train, y_train)

In [ ]:
X_train.isnull().sum()

Using Iterative imputation

In [ ]:

def iterativeImputation(X, y):

    #evaluate strategy on our dataset
    results = []

    strategies = ['ascending', 'descending', 'roman', 'arabic', 'random']
    for s in strategies:
        pipeline = Pipeline(steps=[('i', IterativeImputer(imputation_order=s)),('m',RandomForestClassifier())])


        #evaluate model 
        cv = RepeatedStratifiedKFold(n_splits = 10,  n_repeats=3, random_state= 1)
        scores = cross_val_score(pipeline, X , y ,scoring='accuracy', cv=cv, n_jobs=-1)

        #store results 
        results.append(scores)
        print('>%s %.3f (%.3f)'% (s, np.mean(scores), np.std(scores)))
        
iterativeImputation(X_train, y_train)

Removing columns with extremely low variance

In [ ]:
X = dfHE.drop(columns=['area'])
y = dfHE['area']

results = []
# thresholds = np.arange(0.0, 0.5, 0.05)

# for t in thresholds:
#     transform = VarianceThreshold(threshold=t)
#     print(X.shape)
#     X_sel = transform.fit_transform(X)
#     print("Threshold {} n_features {}".format(t, X_sel.shape[1]))
#     print(X_sel)
#     results.append(X_sel.shape[1])
    
transform = VarianceThreshold(threshold=0.125)
transform.fit(X)
# print("Threshold {} n_features {}".format(t, X_sel.shape[1]))
X_sel = X[X.columns[transform.get_support(indices=True)]]
X_sel

In [ ]:
y_train = data[:, 24]

In [ ]:
# Libraries and configurations for figure plotting
plt.style.use('seaborn')
df.hist(bins=30, figsize=(20,15)) # plotting the histogram


In [ ]:
#checking for the occurences of fires in certain locations 
df.plot(kind='scatter', x='X', y='Y', alpha=0.1, s=300) 

In [ ]:
# plotting the graphs by increasing the size to see the affect of area over the datapoints
df.plot(kind='scatter', x='X', y='Y', alpha=0.2, s=20*df['area']) 

In [ ]:
# Plotting the histogram for the RH attribute
attributes = ['RH']
scatter_matrix(df[attributes], figsize=(15,10))

In [ ]:
# Plotting the histogram for the temp attribute
attributes = ['temp']
scatter_matrix(df[attributes], figsize=(15,10))

In [ ]:
# Plotting the histogram for the DMC attribute

attributes = ['DMC']
scatter_matrix(df[attributes], figsize=(15,10))

In [ ]:


# analyzing categorical columns
# graphing the distribution of the fires area
plt.figure(figsize=(16,5))
print("Skew: {}".format(df['area'].skew()))
print("Kurtosis: {}".format(df['area'].kurtosis()))
ax = sns.kdeplot(df['area'],shade=True,color='g')
plt.xticks([i for i in range(0,1200,50)])
plt.show()

In [ ]:
dfa = df.drop(columns='area')
cat_columns = dfa.select_dtypes(include='object').columns.tolist()
num_columns = dfa.select_dtypes(exclude='object').columns.tolist()

# analyzing categorical columns
plt.figure(figsize=(16,10))
for i,col in enumerate(cat_columns,1):
    plt.subplot(2,2,i)
    sns.countplot(data = dfa,y = col)
    plt.subplot(2,2,i+2)
    df[col].value_counts(normalize=True).plot.bar()
    plt.ylabel(col)
    plt.xlabel('% distribution per category')
plt.tight_layout()
plt.show() 

In [ ]:
# Correlation analysis of the dataset
df.corr()

In [ ]:
df = df.drop(columns=['X', 'Y'])

months_to_remove = ['nov','jan','may']
df = df.drop(df[df.month.isin(months_to_remove)].index ,axis=0)

df['temp_bins'] = pd.cut(df.temp, bins=[0, 15, 20, 25, 40], include_lowest=True, 
                                 labels=['0-15', '15-20', '20-25', '>25'])

# split into train/test
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_index, test_index in split.split(df.values, df.temp_bins.values):
    st_train_set = df.iloc[train_index]
    st_test_set = df.iloc[test_index]


In [ ]:
X_train = st_train_set.drop(columns=['area', 'temp_bins'])
X_train = pd.get_dummies(X_train)
y_train = st_train_set[['area']]

X_test = st_test_set.drop(columns=['area', 'temp_bins'])
X_test = pd.get_dummies(X_test)
y_test = st_test_set[['area']]

model = RandomForestRegressor(max_depth=2, min_samples_split=2, min_samples_leaf=1, random_state=0, n_estimators=100)
model.fit(X_train, y_train.values.ravel())

y_pred = model.predict(X_test)

# print out the prediction scores
print('RMSE: {}'.format(np.sqrt(mean_squared_error(y_test, y_pred))))
print('MAE: {}'.format(mean_absolute_error(y_test, y_pred)))
print('R-squared: {}'.format(r2_score(y_test, y_pred)))



We see that the model performs bad. Ideally the dots should be near the line.
That's because of the numerous 0 values of our output variable, our model is trained to predict only low area values. We need to resample the dataset in the way our model will be able to learn to predict high values of the target variable because they are of interest.

**
Addressing Extreme Rare Cases with SmoteR and SMOGN for Regression**

# Implement SmoteR

Let's start with SmoteR: a simple algorithm to generate more examples of extreme rare cases, we want to learn to predict. 
We will implement it referencing the paper:https://www.researchgate.net/publication/220699419_Utility-Based_Regression

In [ ]:
# define sigmoid function
# https://en.wikipedia.org/wiki/Sigmoid_function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# plot sigmoid function
plt.plot(np.linspace(-10, 10, 100), sigmoid(np.linspace(-10, 10, 100)))
plt.title('Sigmoid Function')
plt.xlabel('x')
plt.axhline(y=0, c='black', linestyle=':')
plt.axvline(x=0, c='black', linestyle=':')
plt.ylabel('sigmoid(x)')
plt.show()



In [ ]:
# implement relevance function
def relevance(x):
    x = np.array(x)
    return sigmoid(x - 50)

# plot relevance function
plt.plot(np.linspace(30, 70, 1000), relevance(np.linspace(30, 70, 1000)))
plt.title('Relevance Function')
plt.xlabel('x')
plt.axhline(y=0, c='gray', linestyle='--')
plt.axvline(x=50, c='gray', linestyle='--')
plt.ylabel('relevance(x)')
plt.show()

Now we implement SmoteR:

In [ ]:
# implement SMOTER
# see paper: https://core.ac.uk/download/pdf/29202178.pdf

def get_synth_cases(D, target, o=200, k=3, categorical_col = []):
    '''
    Function to generate the new cases.
    INPUT:
        D - pd.DataFrame with the initial data
        target - string name of the target column in the dataset
        o - oversampling rate
        k - number of nearest neighbors to use for the generation
        categorical_col - list of categorical column names
    OUTPUT:
        new_cases - pd.DataFrame containing new generated cases
    '''
    new_cases = pd.DataFrame(columns = D.columns) # initialize the list of new cases 
    ng = o // 100 # the number of new cases to generate
    for index, case in D.iterrows():
        # find k nearest neighbors of the case
        knn = KNeighborsRegressor(n_neighbors = k+1) # k+1 because the case is the nearest neighbor to itself
        knn.fit(D.drop(columns = [target]).values, D[[target]])
        neighbors = knn.kneighbors(case.drop(labels = [target]).values.reshape(1, -1), return_distance=False).reshape(-1)
        neighbors = np.delete(neighbors, np.where(neighbors == index))
        for i in range(0, ng):
            # randomly choose one of the neighbors
            x = D.iloc[neighbors[np.random.randint(k)]]
            attr = {}          
            for a in D.columns:
                # skip target column
                if a == target:
                    continue;
                if a in categorical_col:
                    # if categorical then choose randomly one of values
                    if np.random.randint(2) == 0:
                        attr[a] = case[a]
                    else:
                        attr[a] = x[a]
                else:
                    # if continious column
                    diff = case[a] - x[a]
                    attr[a] = case[a] + np.random.randint(2) * diff
            # decide the target column
            new = np.array(list(attr.values()))
            d1 = cosine_similarity(new.reshape(1, -1), case.drop(labels = [target]).values.reshape(1, -1))[0][0]
            d2 = cosine_similarity(new.reshape(1, -1), x.drop(labels = [target]).values.reshape(1, -1))[0][0]
            attr[target] = (d2 * case[target] + d1 * x[target]) / (d1 + d2)
            
            # append the result
            new_cases = new_cases.append(attr,ignore_index = True)
                    
    return new_cases

def SmoteR(D, target, th = 0.999, o = 200, u = 100, k = 3, categorical_col = []):
    '''
    The implementation of SmoteR algorithm:
    https://core.ac.uk/download/pdf/29202178.pdf
    INPUT:
        D - pd.DataFrame - the initial dataset
        target - the name of the target column in the dataset
        th - relevance threshold
        o - oversampling rate
        u - undersampling rate
        k - the number of nearest neighbors
    OUTPUT:
        new_D - the resulting new dataset
    '''
    # median of the target variable
    y_bar = D[target].median()
    
    # find rare cases where target less than median
    rareL = D[(relevance(D[target]) > th) & (D[target] > y_bar)]  
    # generate rare cases for rareL
    new_casesL = get_synth_cases(rareL, target, o, k , categorical_col)
    
    # find rare cases where target greater than median
    rareH = D[(relevance(D[target]) > th) & (D[target] < y_bar)]
    # generate rare cases for rareH
    new_casesH = get_synth_cases(rareH, target, o, k , categorical_col)
    
    new_cases = pd.concat([new_casesL, new_casesH], axis=0)
    
    # undersample norm cases
    norm_cases = D[relevance(D[target]) <= th]
    # get the number of norm cases
    nr_norm = int(len(norm_cases) * u / 100)
    
    norm_cases = norm_cases.sample(min(len(D[relevance(D[target]) <= th]), nr_norm))
    
    # get the resulting dataset
    new_D = pd.concat([new_cases, norm_cases], axis=0)
    
    return new_D

# Add SmoteR

In [ ]:
# construct the initial dataset for SmoteR
cols = X_train.columns.tolist()
cols.append('area')
D = pd.DataFrame(np.concatenate([X_train, y_train], axis=1), columns = cols)

In [ ]:
# apply SmoteR to get the new dataset
np.random.seed(43)
Xs = SmoteR(D, target='area', th = 0.999, o = 300, u = 100, k = 10, categorical_col = ['month_apr',
       'month_aug', 'month_dec', 'month_feb', 'month_jan', 'month_jul',
       'month_jun', 'month_mar', 'month_may', 'month_nov', 'month_oct',
       'month_sep', 'day_fri', 'day_mon', 'day_sat', 'day_sun', 'day_thu',
       'day_tue', 'day_wed'])

X_train = Xs.drop(columns=['area'])
y_train = Xs[['area']]


# model and check the results
model = RandomForestRegressor(max_depth=2, min_samples_split=5, min_samples_leaf=5, random_state=0, n_estimators=100)

model.fit(X_train, y_train.values.ravel())

y_pred = model.predict(X_test)

# print out the prediction scores
print('RMSE: {}'.format(np.sqrt(mean_squared_error(y_test, y_pred))))
print('MAE: {}'.format(mean_absolute_error(y_test, y_pred)))
print('R-squared: {}'.format(r2_score(y_test, y_pred)))

In [ ]:
print(D.describe())

In [ ]:
# plot the results
plt.scatter(y_pred, y_test)
plt.plot(np.linspace(0,400,400), np.linspace(0,400,400), c = 'orange', linestyle='--')
plt.xlabel('prediction')
plt.ylabel('true values')
plt.xlim(0,400)
plt.ylim(0,400)
plt.title('Predicted vs True values')

Our metrics haven't improved much. But now we are actually trying to predict something rather than zero.


Implement SMOGN

# Add SMOGN

In [ ]:

import smogn

X_s = smogn.smoter(
    
    ## primary arguments / inputs
    D,                     ## training set  (pandas dataframe)
    'area',                        ## response variable y by name  (string)
    k = 5,                    ## num of neighs for over-sampling  (pos int)
    samp_method = "extreme",  ## % over / under sample  ("balance" or extreme")
    drop_na_col = True,       ## auto drop columns with nan's  (bool)
    drop_na_row = True,       ## auto drop rows with nan's  (bool)
    replace = False,          ## sampling replacement  (bool)
    
    ## phi relevance function arguments / inputs
    rel_thres = 0.80,          ## relevance threshold considered rare  (pos real)
    rel_method = "auto",      ## relevance method  ("auto" or "manual")
    rel_xtrm_type = "high",   ## distribution focus  ("high", "low", "both")
    rel_coef = 2.25,           ## coefficient for box plot  (pos real)
    rel_ctrl_pts_rg = None    ## input for "manual" rel method  (2d array)
    
    )


In [ ]:
res = X_s['area'] ==0
print(len(res))
print(res.sum())
print(res.sum()/len(res))
#Percentage of zero areas has drastically decreased due to resampling of minority into the majority
plt.plot(X_s['area'])

In [ ]:
X_train = X_s.drop(columns=['area'])
y_train = X_s[['area']]


# model and check the results
model = RandomForestRegressor(max_depth=2, min_samples_split=2, min_samples_leaf=1, random_state=0, n_estimators=100)

model.fit(X_train, y_train.values.ravel())

y_pred = model.predict(X_test)

# print out the prediction scores
print('RMSE: {}'.format(np.sqrt(mean_squared_error(y_test, y_pred))))
print('MAE: {}'.format(mean_absolute_error(y_test, y_pred)))
print('R-squared: {}'.format(r2_score(y_test, y_pred)))

In [ ]:
# plot the results
plt.scatter(y_pred, y_test)
plt.plot(np.linspace(0,400,400), np.linspace(0,400,400), c = 'orange', linestyle='--')
plt.xlabel('prediction')
plt.ylabel('true values')
plt.xlim(0,400)
plt.ylim(0,400)
plt.title('Predicted vs True values')

We can now predict non-zero forest fire areas,and we improved our R^2 score which shows the effectiveness of using oversampling for rare cases for our dataset and undersampling the 0 hectares burn area. 

In [ ]:
X_s.describe()

# Handling Missing Values

Now we randomly designate some instances as missing data

In [ ]:
# from numpy.random import default_rng
# rng = default_rng()
# cols_with_missing_data = ['wind', 'RH', 'DMC', 'ISI', 'rain']
missing_indices_rain = rng.choice(len(X_s['rain'])-1, size=int(0.6 * len(X_s['rain'])), replace=False)
missing_indices = list(map(lambda col: rng.choice(len(X_s[col])-1, size=int(0.15 * len(X_s[col])), replace=False), cols_with_missing_data[0:-1]))
# missing_indices.append(missing_indices_rain)

# for col in range(len(cols_with_missing_data)):
#     for missingIndex in missing_indices[col]:
#         X_s[cols_with_missing_data[col]][missingIndex] = np.nan

# X_s.isnull().sum()
X_s


In [ ]:
plt.plot(df['area'])

Using SimpleImputer with all strategies.

In [ ]:

def SimpleImputerImpute(X, y):
    results = []
    strategies = ['mean','median','most_frequent','constant']
    for s in strategies:
        pipeline = Pipeline(steps=[('i', SimpleImputer(strategy=s)),('m',RandomForestClassifier())])


        #evaluate model 
        cv = RepeatedStratifiedKFold(n_splits = 10,  n_repeats=3, random_state= 1)
        scores = cross_val_score(pipeline, X , y ,scoring='accuracy', cv=cv, n_jobs=-1)

        #store results 
        results.append(scores)
        print('>%s %.3f (%.3f)'% (s, mean(scores), std(scores)))
        return results

data = X_s.values
cols = X_s.columns
ix = [i for i in range(data.shape[1]) if  i != 24]

X_train = data[:, ix]
y_train = data[:, 24]


# from sklearn.utils.multiclass import type_of_target
# type_of_target(df['area'].values)
SimpleImputerImpute(X_train, y_train)

# SimpleImputerImpute(X_train, )

In [ ]:

X_m, y_m = df_ml.drop(columns=['area']), df_ml['area']


In [ ]:


# RFECV is a variant with inbuilt Cross validation
model = LinearRegression()
selector = RFECV(model,cv=5)
selector = selector.fit(X_m, y_m)
print(f"Out of {len(X_m.columns)} features, best number of features {selector.n_features_}")
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score")
plt.plot(range(1, len(X_m.columns) + 1), selector.grid_scores_)
plt.show()



In [ ]:
dfHE = pd.get_dummies(df)

X = dfHE.drop(columns=['area'])
y = dfHE['area']
X= X.values
y= y.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)


fs = SelectKBest(score_func=f_regression, k='all')
# learn relationship from training data
fs.fit(X_train, y_train)
# transform train input data
X_train_fs = fs.transform(X_train)
# transform test input data
X_test_fs = fs.transform(X_test)
# print(X_train_fs, X_test_fs, fs, sep="\n")
for i in range(len(fs.scores_)):
    print('Feature %d: %f' % (i, fs.scores_[i]))
    
# plot the scores
plt.bar([i for i in range(len(fs.scores_))], fs.scores_)
plt.show()



In [ ]:
dfHE.columns[26]

In [ ]:
X = dfHE.drop(columns=['area'])
y = dfHE['area']

results = []
thresholds = np.arange(0.0, 0.5, 0.05)

# for t in thresholds:
#     transform = VarianceThreshold(threshold=t)
#     print(X.shape)
#     X_sel = transform.fit_transform(X)
#     print("Threshold {} n_features {}".format(t, X_sel.shape[1]))
#     print(X_sel)
#     results.append(X_sel.shape[1])
    
transform = VarianceThreshold(threshold=0.15)
transform.fit(X)
# print("Threshold {} n_features {}".format(t, X_sel.shape[1]))
X_sel = X[X.columns[transform.get_support(indices=True)]]
X_sel

# results

# Examining Clustering patterns using KMEANs  

In [ ]:

kmeans = KMeans(init="random",n_clusters=3,n_init=10,max_iter=300,random_state=42)

m = kmeans.fit(X_train.values)
#print(m)
# The lowest SSE value
lowest = kmeans.inertia_
#print(lowest)

# Final locations of the centroid
centroid = kmeans.cluster_centers_
#print(centroid)

# The number of iterations required to converge
converge = kmeans.n_iter_


l = kmeans.labels_[:5]

kmeans_kwargs = {"init": "random", "n_init": 10, "max_iter": 300, "random_state": 42, }

sse = []
for k in range(1, 11):
     kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
     kmeans.fit(X_train.values)
     sse.append(kmeans.inertia_)

plt.style.use("fivethirtyeight")
plt.plot(range(1, 11), sse)
plt.xticks(range(1, 11))
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.show()